In [1]:
import asyncio
import time
import requests
from indy import anoncreds, did, ledger, pool, wallet, IndyError

import json


async def run():
    print("Supplier -> started")

    # Set protocol version 2 to work with Indy Node 1.4
    await pool.set_protocol_version(2)

    pool_ = {
        'name': 'pool1',
        'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
    }
    print("Open Pool Ledger: {}".format(pool_['name']))

    try:
        await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
    except IndyError as ex:
        if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
            pass
    pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

    print("==============================")
    print("== Getting Trust Anchor credentials - Supplier getting Verinym  ==")
    print("------------------------------")

    supplier = {
        'name': 'Supplier',
        'wallet_config': json.dumps({'id': 'supplier_wallet'}),
        'wallet_credentials': json.dumps({'key': 'supplier_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'TRUST_ANCHOR'
    }

    response = requests.post('http://steward:8888/get_verinym', data = {'entity': supplier})
    supplier = response['entity']

    # await getting_verinym(steward, supplier)

    print("Close and Delete pool")
    await pool.close_pool_ledger(pool_['handle'])
    await pool.delete_pool_ledger_config(pool_['name'])

    print("Getting started -> done")


async def create_wallet(identity):
    print("\"{}\" -> Create wallet".format(identity['name']))
    try:
        await wallet.create_wallet(identity['wallet_config'], identity['wallet_credentials'])
    except IndyError as ex:
        if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
            pass
    identity['wallet'] = await wallet.open_wallet(identity['wallet_config'], identity['wallet_credentials'])


async def getting_verinym(from_, to):
    await create_wallet(to)

    (to['did'], to['key']) = await did.create_and_store_my_did(to['wallet'], "{}")

    from_['info'] = {
        'did': to['did'],
        'verkey': to['key'],
        'role': to['role'] or None
    }

    await send_nym(from_['pool'], from_['wallet'], from_['did'], from_['info']['did'],
                   from_['info']['verkey'], from_['info']['role'])


async def send_nym(pool_handle, wallet_handle, _did, new_did, new_key, role):
    nym_request = await ledger.build_nym_request(_did, new_did, new_key, None, role)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, nym_request)


if __name__ == '__main__':
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    await run()
    time.sleep(1)  # FIXME waiting for libindy thread complete


Getting started -> started
Open Pool Ledger: pool1
=== Getting Trust Anchor credentials for Faber, Acme, Thrift and Government  ==
------------------------------
"Sovrin Steward" -> Create wallet
"Sovrin Steward" -> Create and store in Wallet DID from seed
== Getting Trust Anchor credentials - Government getting Verinym  ==
------------------------------
"Government" -> Create wallet
== Getting Trust Anchor credentials - Faber getting Verinym  ==
------------------------------
"Faber" -> Create wallet
== Getting Trust Anchor credentials - Acme getting Verinym  ==
------------------------------
"Acme" -> Create wallet
== Getting Trust Anchor credentials - Thrift getting Verinym  ==
------------------------------
"Thrift" -> Create wallet
=== Credential Schemas Setup ==
------------------------------
"Government" -> Create "Job-Certificate" Schema
"Government" -> Send "Job-Certificate" Schema to Ledger
"Government" -> Create "Transcript" Schema
"Government" -> Send "Transcript" Schema to